# Data Pre Processing

In [249]:
%load_ext autoreload
%autoreload 2

from utils import code
from adjust_datatype import urlify, int_to_float, pct_to_numeric
from adjust_datatype import string_to_numeric, string_to_datetime, currency_to_numeric
from plot_libraries import setup_graphics

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [250]:
# load libraries and set plot parameters
import os, random, re, sys, time, warnings
import math
import numpy as np
import pandas as pd
import pandas_profiling
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

%matplotlib inline
sns.set()
pd.options.display.max_columns = None
setup_graphics()

##### Columns Description

### Upload Dataset

In [251]:
dataset = pd.read_csv('../data/dataset_csv.csv', delimiter=';', low_memory=False)
dataset.shape

(17553, 38)

In [252]:
dataset.head()

,Account ID,Number Of Opportunities,Nb Won Opp,Number of Open Opportunities,LastModifiedOpportunitiesDate,Last Activity,Last Lead Activity,Activities Last 30 Days,Employees,ZoomInfo Employee Range,ZoomInfo Revenue Range,Annual Revenue,Organic Visits,Pct Organic Visits,SEO team in the enterprise organization?,SEO Visits,Combined Pages,URLs Indexed,ZoomInfo Global HQ Country,Account Status,Active Service Package,Annual Revenue (converted),Billing Country (text only),Commited MRR - Today (converted),Industry,Adjusted Industry,NbUsers,NbProjects,NbSessions,Net Promoter Score,Account ICP Score,Account ICP Tier,ICP TIER - HS,Page Count,Page Count - Range,Alexa Rank,Became a Customer date,Parent Account Status
0,0012400000L5cmZ,7,1,1,2/14/2020 3:00 PM,11/12/2019,10/2/2019,0,10,-,-,"EUR 3,000,000.0","61,688,430",34.00%,-,"61,688,430",0,"27,700,000",-,Lost Customer,No Service Package,"USD 3,333,900.00",Turkey,USD 0.00,Internet Software & Services,Retail,2,9,9,-,91.667,Tier A,Tier A,"27,700,000",>1M,331,2/12/2020,-
1,0011p00001Sgf9s,0,0,0,-,-,-,0,"2,940","1,000 - 5,000",-,"GBP 1,000,000,000.00",-,-,-,0,"2,670,000",-,United States,Prospect,No Service Package,"USD 1,311,577,953.50",United States,USD 0.00,PUBLISHING,Publisher,-,-,-,-,41.667,Tier A,Tier C - Incomplete,-,-,511,-,-
2,00124000004sEH5,11,8,1,2/14/2020 1:02 PM,2/14/2020,-,51,"10,000",-,-,"EUR 12,000,000,000.0","19,397,082",93.00%,-,"28,615,923","206,300","76,200",-,Active Customer,Service Package Custom,"USD 13,335,600,000.00",France,"USD 1,000.17",Retailing,Retail,7,7,4,8,100,Tier A,Tier A,"206,300",Between 100K and 250K,"8,881",2/3/2020,Prospect
3,00124000015MJ24,0,0,0,-,-,-,0,"10,000",-,-,"EUR 1,000,000,000.0","31,310,211",66.00%,-,"31,310,211","1,570,000","1,570,000",-,Prospect,No Service Package,"USD 1,111,300,000.00",India,USD 0.00,Media,Media,-,-,-,-,100,Tier A,Tier A,"1,570,000",>1M,904,-,Prospect
4,0011p00002dLLfO,0,0,0,-,1/13/2020,-,0,"10,000","Over 10,000",Over $5 bil.,"USD 1,000,000,000.00",-,-,-,0,0,-,United States,Partner,No Service Package,"USD 1,000,000,000.00",United States,USD 0.00,Computer Software,-,-,-,-,-,20.833,Tier A,Tier D - Incomplete,-,-,"1,000,001",-,Prospect


##### Fix column names with whitespaces to underscores

In [253]:
dataset.rename(urlify, axis='columns', inplace=True)

##### Fix column types

In [254]:
dataset.dtypes

Account_ID                                 object
Number_Of_Opportunities                     int64
Nb_Won_Opp                                  int64
Number_of_Open_Opportunities                int64
LastModifiedOpportunitiesDate              object
Last_Activity                              object
Last_Lead_Activity                         object
Activities_Last_30_Days                    object
Employees                                  object
ZoomInfo_Employee_Range                    object
ZoomInfo_Revenue_Range                     object
Annual_Revenue                             object
Organic_Visits                             object
Pct_Organic_Visits                         object
SEO_team_in_the_enterprise_organization    object
SEO_Visits                                 object
Combined_Pages                             object
URLs_Indexed                               object
ZoomInfo_Global_HQ_Country                 object
Account_Status                             object


##### Fix datatypes to numeric

In [255]:
dataset = int_to_float(dataset)

In [256]:
dataset['Pct_Organic_Visits']  = pct_to_numeric(dataset, 'Pct_Organic_Visits')

In [257]:
numeric_cols =['Activities_Last_30_Days', 'Employees', 
               'Organic_Visits', 'SEO_Visits', 'Combined_Pages',
               'URLs_Indexed', 'NbProjects', 
               'NbUsers', 'NbSessions', 'Net_Promoter_Score', 
               'Account_ICP_Score', 'Page_Count', 'Alexa_Rank']

In [258]:
for col in numeric_cols:
    dataset[col] = string_to_numeric(dataset, col)

In [259]:
datetime_cols = ['LastModifiedOpportunitiesDate', 'Last_Activity',
                  'Last_Lead_Activity', 'Became_a_Customer_date' ]

In [260]:
for col in datetime_cols:
    dataset[col] = string_to_datetime(dataset, col)

In [261]:
currency_cols = ['Commited_MRR_Today_converted', 'Annual_Revenue_converted']
for col in currency_cols:
    dataset[col] = currency_to_numeric(dataset, col)

### Target Columns

Our task is to predict if the lead will convert at next try

In [264]:
dataset = dataset[dataset['Account_Status'] != 'Partner'].copy()

In [265]:
def target(status, n_try):
    if status == 'Active Customer': return 'yes'     
    elif status == 'Lost Customer': return 'no'       
    elif (status == 'Prospect') & (n_try >0): return 'no'  
    else: return 'prospect'

In [266]:
dataset['Convert_Next_Contact'] = dataset.apply(lambda x: target(x['Account_Status'],x['Number_Of_Opportunities']),axis=1)

At this time some columns does not have any predictive value, specially beacuse they are generated after the deal.

In [267]:
uniques = dataset[dataset.Convert_Next_Contact == 'prospect'].nunique()
uniques[uniques==1]

Number_Of_Opportunities         1
Nb_Won_Opp                      1
Number_of_Open_Opportunities    1
Account_Status                  1
Active_Service_Package          1
Commited_MRR_Today_converted    1
Convert_Next_Contact            1
dtype: int64

 We have to drop them (except our target variable)

In [268]:
drop_uniques = uniques[uniques==1][:-1].index

In [269]:
dataset.drop(columns=drop_uniques, inplace=True)

### Fix Critic Errors

##### We must drop some columns with critical problems such as:
* `SEO_team_in_the_enterprise_organization:` Only 7 rows filled 

In [278]:
dataset.SEO_team_in_the_enterprise_organization.value_counts()

-                                 14647
Under marketing                       4
Under the website product team        3
Name: SEO_team_in_the_enterprise_organization, dtype: int64

In [279]:
dataset.drop(columns=['SEO_team_in_the_enterprise_organization'], inplace=True)

##### Some rows are highly empty, most of them are tagged as 'Incomplete' (+-20%), in the `Account_ICP_Tier` column. Let's remove them.

In [276]:
dataset = dataset[dataset['Account_ICP_Tier'] != 'Incomplete'].copy()

### Split Dataset

In [280]:
trainDF = dataset[dataset['y'].isin(['yes' , 'no'])].copy()

In [281]:
PredDF = dataset[dataset['y'] == 'prospect'].copy()

In [282]:
trainDF.to_csv(r'/Users/nicholasrichers/Documents/GitHub/b2b_sales_project/data/trainDF.csv', header=True)

In [283]:
PredDF.to_csv(r'/Users/nicholasrichers/Documents/GitHub/b2b_sales_project/data/PredDF.csv', header=True)

In [285]:
trainDF.shape

(1849, 32)

# Exploratory Data Analisys

In [ ]:
X, y = get_data('../data/train.csv')
X.head()

In [ ]:
dtypes = pd.DataFrame(dataset.dtypes.rename('type')).reset_index().astype('str')
numeric = dtypes[(dtypes.type.isin(['int64', 'float64']))]['index'].values
categorical = dtypes[~(dtypes['index'].isin(numeric))]['index'].values

##### Drop Columns
* `Account_ID`: Index Column
* `LastModifiedOpportunitiesDate:` Similar to 'Activities_Last_30_Days'
* `Last_Activity:` Similar to 'Activities_Last_30_Days'
* `Last_Lead_Activity:` Similar to 'Activities_Last_30_Days'
* `ZoomInfo_Employee_Range:` Similar to 'Activities_Last_30_Days'
* `ZoomInfo_Revenue_Range:` Similar to 'Annual_Revenue_converted'
* `Annual_Revenue:` Similar to 'Annual_Revenue_converted'
* `ZoomInfo_Global_HQ_Country:` Categorical (+100 distinct)
* `Billing_Country_text_only:`  Categorical (+100 distinct)
* `Industry:` Categorical (+100 distinct)
* `Page_Count_Range:` Similar to 'Page_Count'
* `SEO_team_in_the_enterprise_organization:` Only 7 rows filled 

* `Account_ICP_Tier:` Similar to 'Account_ICP_Score'
* `ICP_TIER_HS:` Similar to 'Account_ICP_Score'
* `URLs_Indexed:` High Correlated to 'Page_count'




In [ ]:
dataset.set_index(['Account_ID'], inplace=True)

In [ ]:
drop_columns = ['LastModifiedOpportunitiesDate', 'Last_Activity',
                'Last_Lead_Activity', 'URLs_Indexed',
                'ZoomInfo_Employee_Range', 'ZoomInfo_Revenue_Range',
                'ZoomInfo_Global_HQ_Country', 'Annual_Revenue',
                'Billing_Country_text_only', 'Industry', 'Page_Count_Range',
                'SEO_team_in_the_enterprise_organization',
                'Account_ICP_Tier', 'ICP_TIER_HS' ]

In [ ]:
dataset.drop(columns=drop_columns, inplace=True)

In [ ]:
dataset.drop(columns=leaky_variables, inplace=True)

##### Adjusted industry column

In [ ]:
dataset['Adjusted_Industry'].value_counts()

In [ ]:
for val in ['Publisher', '-', 'Other', 'Botify Ltd.']:
    dataset['Adjusted_Industry'].mask(dataset['Adjusted_Industry'] == val, 'Other', inplace=True)

### Missing Values

In [ ]:
nan_mean = dataset.isna().mean()
nan_mean = nan_mean[nan_mean != 0].sort_values()
nan_mean

##### Fill NA Methods
* **Alexa_Rank:** MAX
* **Account_ICP_Score:** MEDIAN (industry)
* **Employees:** MEDIAN (industry)
* **Annual_Revenue_converted:** MEDIAN (industry)
* **Organic_Visits:** MIN
* **Page_Count:** MIN
* **Pct_Organic_Visits:**  MIN
* **Combined_Pages:** DROP

In [ ]:
fill_max = ['Alexa_Rank' ]
fill_min = ['Organic_Visits', 'Page_Count', 'Pct_Organic_Visits']
fill_median = ['Account_ICP_Score', 'Employees', 'Annual_Revenue_converted' ]
drop_missing_values = ['Combined_Pages']

In [ ]:
dataset[fill_max] = dataset[fill_max].fillna(dataset[fill_max].max())
dataset[fill_min] = dataset[fill_min].fillna(dataset[fill_min].min())
dataset.drop(columns=drop_missing_values, inplace=True)

In [ ]:
values_dict = dataset.groupby(['Adjusted_Industry'])[fill_median].median().to_dict()
for col in fill_median:
    dataset[col] = dataset[col].fillna(dataset['Adjusted_Industry'].map(values_dict[col]))

### Correlated Columns

* **Organic_Visits** & **SEO_Visits** are considered high correlated, I will drop **Organic_Visits**

In [ ]:
#dataset.profile_report()

In [ ]:
dataset.drop(columns=['Organic_Visits'], inplace=True)

### Final Version

In [ ]:
dataset.head()

In [ ]:
dataset.dtypes